# Finding our best-performing salespeople and products

## Introduction

**Business Context.** You work for AdventureWorks, a company that sells outdoor sporting equipment. The company has many different locations and has been recording the sales of different locations on various products. You, their new data scientist, have been tasked with the question: *What are our best products and salespeople and how can we use this information to improve our overall performance?*

You have been given access to the relevant data files with documentation from the IT department. Your job is to extract meaningful insights from these data files to help increase sales. First, you will look at the best products and try to see how different products perform in different categories. Second, you will analyze the best salespeople to see if the commission percentage motivates them to sell more.

**Business Problem.** Your task is to *write queries in SQL to carry out the requested analysis*.

**Analytical Context.** You are given the data as a SQLite database. The company has been pretty vague about how they expect you to extract insights, but you have come up with the following plan of attack:

1. Load the database and ensure you can run basic queries against it
2. Look at how product ratings and total sales are related
3. See how products sell in different subcategories (bikes, helmets, socks, etc.)
4. Calculate which salespeople have performed the best in 2014
5. See if total sales are correlated with commission percentage

Of course, this is only your initial plan. As you explore the database, your strategy will likely change.

## Overview of the data

The data for this case is contained in the [`AdventureWorks.db`](/extended.sql_fellow/files/AdventureWorks.db) SQLite database. We will be focusing on the tables that belong to the Sales and Product categories. Complete documentation, with schemas, for the original data (of which you have only a subset) can be found [here](https://dataedo.com/download/AdventureWorks.pdf).

**Product Tables (Pg. 34 in documentation):**
* **Product**: one row per product that the company sells
* **ProductReview**: one row per rating and review left by customers
* **ProductModelProductDescriptionCulture**: a link between products and their longer descriptions also indicating a "culture" - which language and region the product is for
* **ProductDescription**: a longer description of each product, for a specific region
* **ProductCategory**: the broad categories that products fit into
* **ProductSubCategory**: the narrower subcategories that products fit into

**Sales Tables (Pg. 71 in documentation):**
* **SalesPerson**: one row per salesperson, including information on their commission and performance
* **SalesOrderHeader**: one row per sale summarizing the sale
* **SalesOrderDetail**: many rows per sale, detailing each product that forms part of the sale
* **SalesTerritory**: the different territories where products are sold, including performance
* **CountryRegionCurrency**: the currency used by each region
* **CurrencyRate**: the average and closing exchange rates for each currency compared to the USD

**Tip**: Review the documentation carefully to learn more about the tables (like relevant columns in each) and the relationships between them. Note that not all columns may be available in the subset provided in this case as they are not necessary for the following exercises. 

Importing the libraries:

In [ ]:
import pandas as pd
import sqlite3

Let's now load in the database:

In [ ]:
cxn = sqlite3.connect('AdventureWorks.db')

To run SQL queries from within this notebook, you should define a variable that includes the SQL statement at the start of your code cell, like this:

In [ ]:
# Name your variable five_products
five_products = """
    SELECT * FROM product LIMIT 5;
"""

pd.read_sql(five_products, cxn)

,productid,NAME,productnumber,makeflag,finishedgoodsflag,color,safetystocklevel,reorderpoint,standardcost,listprice,...,productline,class,style,productsubcategoryid,productmodelid,sellstartdate,sellenddate,discontinueddate,rowguid,modifieddate
0,1,Adjustable Race,AR-5381,f,f,,1000,750,0.0,0.0,...,,,,,,2008-04-30 00:00:00,,,694215b7-08f7-4c0d-acb1-d734ba44c0c8,2014-02-08 10:01:36.827
1,2,Bearing Ball,BA-8327,f,f,,1000,750,0.0,0.0,...,,,,,,2008-04-30 00:00:00,,,58ae3c20-4f3a-4749-a7d4-d568806cc537,2014-02-08 10:01:36.827
2,3,BB Ball Bearing,BE-2349,t,f,,800,600,0.0,0.0,...,,,,,,2008-04-30 00:00:00,,,9c21aed2-5bfa-4f18-bcb8-f11638dc2e4e,2014-02-08 10:01:36.827
3,4,Headset Ball Bearings,BE-2908,f,f,,800,600,0.0,0.0,...,,,,,,2008-04-30 00:00:00,,,ecfed6cb-51ff-49b5-b06c-7d8ac834db8b,2014-02-08 10:01:36.827
4,316,Blade,BL-2036,t,f,,800,600,0.0,0.0,...,,,,,,2008-04-30 00:00:00,,,e73e9750-603b-4131-89f5-3dd15ed5ff80,2014-02-08 10:01:36.827


## Finding our most popular products

The company would like to know which of their products is the most popular among customers. You figure that the average rating given in reviews is correlated with the number of sales of a particular product (that products with higher reviews have more sales).

### Exercise 1 (1 point)

Using the `product` and `productreview` tables, `INNER JOIN` them and rank the products according to their average review rating. Save the SQL code in a string variable called `rating_ranking`. Please make *absolutely sure* to name your variable exactly that or otherwise your answer will not be recorded.

Your output should look like this:

| productid 	| NAME 	| avgrating 	| num_ratings 	|
|-:	|-:	|-:	|-	|
| 709 	| Mountain Bike Socks, M 	| 5.0 	| 1 	|
| ... 	| ... 	| ... 	| ... 	|


In [ ]:
# Name your variable rating_ranking
# YOUR CODE HERE
rating_ranking = """
    SELECT product.productid, 
    NAME, 
    AVG(productreview.rating) AS avgrating, 
    COUNT(productreview.rating) AS num_rating
    FROM product
    INNER JOIN productreview ON product.productid = productreview.productid
    GROUP BY product.productid;
"""

pd.read_sql(rating_ranking, cxn)

,productid,NAME,avgrating,num_rating
0,709,"Mountain Bike Socks, M",5.0,1
1,798,"Road-550-W Yellow, 40",5.0,1
2,937,HL Mountain Pedal,3.0,2


### Exercise 2 

Much to your disappointment, there are only three products with ratings and only four reviews in total! This is nowhere near enough to perform an analysis of the correlation between reviews and total sales. Since we cannot infer the most popular products from the reviews, we will go with an alternative strategy.

#### 2.1 (1 point)

Get the product model ID and description for each product. Include only descriptions for which `productmodelproductdescriptionculture.cultureid = 'en'`.

Your output should look like this:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>productmodelid</th>      <th>description</th>    </tr>  </thead>  <tbody>    <tr>      <td>1</td>      <td>Light-weight, wind-resistant, packs to fit into a pocket.</td>    </tr>    <tr>      <td>2</td>      <td>Traditional style with a flip-up brim; one-size fits all.</td>    </tr>    <tr>      <td>3</td>      <td>Synthetic palm, flexible knuckles, breathable mesh upper. Worn by the AWC team riders.</td>    </tr>    <tr>      <td>...</td>      <td>...</td>    </tr>  </tbody></table>

In [ ]:
# Name your variable productmodelid_description
# YOUR CODE HERE
productmodelid_description = """
    SELECT productmodelproductdescriptionculture.productmodelid, 
    productdescription.description 
    FROM productmodelproductdescriptionculture
    JOIN productdescription ON productmodelproductdescriptionculture.productdescriptionid = productdescription.productdescriptionid
    WHERE productmodelproductdescriptionculture.cultureid = 'en'
"""

pd.read_sql(productmodelid_description, cxn)

,productmodelid,description
0,1,"Light-weight, wind-resistant, packs to fit int..."
1,2,Traditional style with a flip-up brim; one-siz...
2,3,"Synthetic palm, flexible knuckles, breathable ..."
3,4,"Full padding, improved finger flex, durable pa..."
4,5,Each frame is hand-crafted in our Bothell faci...
...,...,...
122,123,Replacement mountain wheel for entry-level rider.
123,124,Replacement mountain wheel for the casual to s...
124,125,High-performance mountain replacement wheel.
125,126,Replacement road rear wheel for entry-level cy...


### Exercise 3

To get a better sense of the sales, let's look at the correlation between quantity sold and price for each subcategory.

#### 3.1 (1 point)

Write a query that shows how many items were ordered in total for every product in the database. Do not filter by culture.

Your output should look like this:

| productid 	| quantity 	|
|-:	|-:	|
| 707 	| 6266 	|
| 708 	| 6532 	|
| 709 	| 1107 	|
| 710 	| 90 	|
| 711 	| 6743 	|
| 712 	| 8311 	|
| 713 	| 429 	|
| 714 	| 3636 	|
| ... 	| ... 	|

**Hint:** Use the `salesorderdetail` table.

In [ ]:
# Name your variable quantities_ordered
# YOUR CODE HERE
quantities_ordered = """
SELECT salesorderdetail.productid, 
SUM(salesorderdetail.orderqty) AS quantity
FROM salesorderdetail
GROUP BY salesorderdetail.productid
"""

pd.read_sql(quantities_ordered, cxn)

,productid,quantity
0,707,6266
1,708,6532
2,709,1107
3,710,90
4,711,6743
...,...,...
261,994,378
262,996,543
263,997,656
264,998,1556


#### 3.2 (1 point)

Write a query that shows the list price for each product, alongside its category and subcategory. Your output should look like this:

| productid 	| category 	| subcategory 	| listprice 	|
|-:	|-:	|-:	|-:	|
| 680 	| Components 	| Road Frames 	| 1431.5 	|
| 706 	| Components 	| Road Frames 	| 1431.5 	|
| 707 	| Accessories 	| Helmets 	| 34.99 	|
| 708 	| Accessories 	| Helmets 	| 34.99 	|
| 709 	| Clothing 	| Socks 	| 9.5 	|
| 710 	| Clothing 	| Socks 	| 9.5 	|
| 711 	| Accessories 	| Helmets 	| 34.99 	|
| 712 	| Clothing 	| Caps 	| 8.99 	|
| 713 	| Clothing 	| Jerseys 	| 49.99 	|
| 714 	| Clothing 	| Jerseys 	| 49.99 	|
| 715 	| Clothing 	| Jerseys 	| 49.99 	|
| 716 	| Clothing 	| Jerseys 	| 49.99 	|
| 717 	| Components 	| Road Frames 	| 1431.5 	|
| 718 	| Components 	| Road Frames 	| 1431.5 	|
| 719 	| Components 	| Road Frames 	| 1431.5 	|
| ... 	| ... 	| ... 	| ... 	|



In [ ]:
# Name your variable products_prices
# YOUR CODE HERE
products_prices = """
SELECT product.productid, productcategory.name AS category, productsubcategory.name AS subcategory, product.listprice
FROM product
INNER JOIN productsubcategory ON product.productsubcategoryid = productsubcategory.productsubcategoryid
INNER JOIN productcategory ON productsubcategory.productcategoryid = productcategory.productcategoryid;

"""

pd.read_sql(products_prices, cxn)

,productid,category,subcategory,listprice
0,680,Components,Road Frames,1431.50
1,706,Components,Road Frames,1431.50
2,707,Accessories,Helmets,34.99
3,708,Accessories,Helmets,34.99
4,709,Clothing,Socks,9.50
...,...,...,...,...
290,995,Components,Bottom Brackets,101.24
291,996,Components,Bottom Brackets,121.49
292,997,Bikes,Road Bikes,539.99
293,998,Bikes,Road Bikes,539.99


## Finding our top salespeople

As mentioned earlier, we want to find our best salespeople and see whether or not we can incentivize them in an appropriate manner. Namely, we want to determine if the commission percentage we give them motivates them to make more and bigger sales.

### Exercise 4 (1 point)

Find the top five performing salespeople by using the `salesytd` (Sales, year-to-date) column.

Your output should look like this:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>businessentityid</th>      <th>salesytd</th>    </tr>  </thead>  <tbody>    <tr>      <td>276</td>      <td>4251368.5497</td>    </tr>    <tr>      <td>289</td>      <td>4116871.2277</td>    </tr>    <tr>      <td>275</td>      <td>3763178.1787</td>    </tr>    <tr>      <td>...</td>      <td>...</td>    </tr>  </tbody></table>

**Hint:** We only need to know the `businessentityid` for each salesperson as this uniquely identifies each salesperson. Your query should therefore only have two columns: `businessentityid` and `salesytd`.

In [ ]:
# Name your variable salesperson_sales
# YOUR CODE HERE
salesperson_sales = """
SELECT salesperson.businessentityid, 
salesperson.salesytd
FROM salesperson
ORDER BY salesperson.salesytd DESC
LIMIT 5

"""

pd.read_sql(salesperson_sales, cxn)

,businessentityid,salesytd
0,276,4.251369e+06
1,289,4.116871e+06
2,275,3.763178e+06
3,277,3.189418e+06
4,290,3.121616e+06


### Exercise 5 (2 points)

The sales numbers from the previous query are hard-coded into the `salesperson` table, instead of dynamically calculated from each sales record. Currently, we don't know how this number is updated or much about it at all, so it's good to remain skeptical.

Using the ```salesorderheader``` table, find the top 5 salespeople who made the most sales *in the most recent year available* (2014). (There is a column called `subtotal` - use that.) Sales that do not have an associated salesperson should be excluded from your calculations and final output. All orders that were made within the 2014 calendar year should be included.

Your output should look like this:

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>salespersonid</th>      <th>totalsales</th>    </tr>  </thead>  <tbody>    <tr>      <td>289</td>      <td>1382996.5839000002</td>    </tr>    <tr>      <td>276</td>      <td>1271088.5216</td>    </tr>    <tr>      <td>...</td>      <td>...</td>    </tr>  </tbody></table>

**Hint:** You can use the syntax `WHERE column >= '1970-01-01'` to generate an arbitrary date in SQLite and compare this to specific dates in the tables (in this example, dates equal to or later than Jan 1, 1970). Additionally, when you want to make sure that columns with empty or null values are excluded from a query in SQLite, you have to add a line like this one to your `WHERE` statement: `my_column IS NOT NULL AND my_column <> ""`. The `<>` operator is the opposite of `=`, that is, it checks that two values are different from each other.

In [ ]:
# Name your variable salesperson_totalsales
# YOUR CODE HERE
salesperson_totalsales = """
SELECT salesorderheader.salespersonid, SUM(salesorderheader.subtotal) AS totalsales
FROM salesorderheader
WHERE salesorderheader.orderdate  >= '2014-01-01' AND salesorderheader.salespersonid IS NOT NULL AND salesorderheader.salespersonid <> ""
GROUP BY salespersonid
ORDER BY totalsales DESC
LIMIT 5

"""

pd.read_sql(salesperson_totalsales, cxn)

,salespersonid,totalsales
0,289,1.382997e+06
1,276,1.271089e+06
2,275,1.057247e+06
3,282,1.044811e+06
4,277,1.040093e+06


#### 6.1 (1 point)

Write a query that shows for each `salesorderid` (find this column in the `salesorderdetail` table) the total amount of money paid. Remember to subtract `unitpricediscount` from each item's price (`unitpricediscount` is a percentage).

Your output should look like this:

| salesorderid 	| ordertotal 	|
|-:	|-:	|
| 43659 	| 20565.6206 	|
| 43660 	| 1294.2529 	|
| 43661 	| 32726.4786 	|
| 43662 	| 28832.5289 	|
| 43663 	| 419.4589 	|
| 43664 	| 24432.608799999995 	|
| 43665 	| 14352.7713 	|
| 43666 	| 5056.4896 	|
| 43667 	| 6107.081999999999 	|
| 43668 	| 35944.156200000005 	|
| 43669 	| 714.7043 	|
| ... 	| ... 	|

In [ ]:
# Name your variable order_ordertotal
# YOUR CODE HERE
order_ordertotal = """
SELECT salesorderdetail.salesorderid, SUM((salesorderdetail.unitprice * orderqty) * (1 - salesorderdetail.unitpricediscount / 100)) AS ordertotal
FROM salesorderdetail
GROUP BY salesorderdetail.salesorderid;
"""
pd.read_sql(order_ordertotal,cxn)

,salesorderid,ordertotal
0,43659,20565.6206
1,43660,1294.2529
2,43661,32726.4786
3,43662,28832.5289
4,43663,419.4589
...,...,...
31460,75119,42.2800
31461,75120,84.9600
31462,75121,74.9800
31463,75122,30.9700
